In [44]:
import pandas as pd
df = pd.read_csv('Diabetes_Drug-Drug_Interactions.csv')
df = df.rename({"Drug": "Source", "Not Advised With (Specific Drug, no need to list drugs if they belong to a class that's in the previous column)": "Single Drug", "Not Advised With (Class of Drug, put SINGULAR!)": "Adverse Class"}, axis=1)
df

,Class,Source,Alternate Names,Adverse Class,Single Drug,Classification of Risk,Mechanism,Adverse Effect,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Biguanide,Metformin,"Glucophage, Fortamet, Glutetza, Riomet",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Thiazolidinediones (TZD),pioglitazone,Actos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Classification of Risk: Mild, Moderate, Severe"
2,Thiazolidinediones (TZD),rosiglitazone,Avandia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GLP1 mimetics,albiglutide,NaN,Meglitinides,glipizide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
4,GLP1 mimetics,dulaglutide,NaN,Meglitinides,repaglinide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
5,GLP1 mimetics,exenatide,NaN,Meglitinides,glipizide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
6,GLP1 mimetics,liraglutide,victoza,Meglitinides,repaglinide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
7,GLP1 mimetics,lixisenatide,"soliqua, insulin glargine",Meglitinides,glipizide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
8,GLP1 mimetics,semaglutide,ozempic,Meglitinides,repaglinide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
9,DPP4,alogliptin,"Nesina, Vipidia",Meglitinides,NaN,Moderate,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
"""First write out straightforward source - target - label triplets"""
df_singles = df[df['Single Drug'].notna()] # slice dataframe to just see
# need to map from "Adverse Class" to "Class" column and fetch those drugs 
df_singles = df_singles[['Source', 'Single Drug', 'Classification of Risk']]
df_singles.to_csv('./ddis_from_spreadsheet.csv', index=False)

In [54]:
def explode_list(drug_class, drug_class_mappings):
    try:
        return drug_class_mappings[drug_class]
    except:
        return None

In [56]:
"""Now append to csv"""
# for now, ignore classes where there are no drugs
df_classes = df[df['Adverse Class'].notna()]
drug_classes = list(df['Adverse Class'])
drug_classes = [x for x in drug_classes if str(x) != 'nan']
drug_class_mappings = {}
for drug_class in drug_classes:
    drugs = set(df_classes[df_classes['Class'] == drug_class]['Source'])
    if len(drugs) > 0:
        print(drug_class)
        drug_class_mappings[drug_class] = set(df_classes[df_classes['Class'] == drug_class]['Source'])
df_classes['Adverse Drugs'] = df_classes['Adverse Class'].apply(explode_list, drug_class_mappings=drug_class_mappings)
df_classes = df_classes.explode('Adverse Drugs')
df_classes_slice = df_classes[['Source', 'Adverse Drugs', 'Classification of Risk']]
df_classes_slice.to_csv('./ddis_from_spreadsheet.csv', mode='a', header=False, index=False)

Meglitinides
Meglitinides
Meglitinides
Meglitinides
Meglitinides
Meglitinides
Meglitinides
Meglitinides
Meglitinides
Meglitinides
DPP4
DPP4
Sulfonylureas
Sulfonylureas
Sulfonylureas
Sulfonylureas
GLP1 mimetics
GLP1 mimetics


/var/folders/xp/yp2gh__s5j188pt1jsvz_1_r0000gr/T/ipykernel_33182/846326413.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classes['Adverse Drugs'] = df_classes['Adverse Class'].apply(explode_list, drug_class_mappings=drug_class_mappings)


In [57]:
df_classes[df_classes['Source'] == 'alogliptin']

,Class,Source,Alternate Names,Adverse Class,Single Drug,Classification of Risk,Mechanism,Adverse Effect,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Adverse Drugs
9,DPP4,alogliptin,"Nesina, Vipidia",Meglitinides,NaN,Moderate,NaN,NaN,NaN,NaN,NaN,NaN,nateglinide
9,DPP4,alogliptin,"Nesina, Vipidia",Meglitinides,NaN,Moderate,NaN,NaN,NaN,NaN,NaN,NaN,repaglinide
